In [ ]:
!pip install scikit-learn numpy pandas imbalanced-learn optuna --quiet

In [ ]:
import pandas as pd

data = pd.read_csv("datasets/IBM Dataset 1.csv")

data = data.drop(columns=['EmployeeCount', 'EmployeeNumber', 'StandardHours', 'Over18'], errors='ignore')

engineered_features = [
    ('IncomePerJobLevel', lambda df: df['MonthlyIncome'] / (df['JobLevel'] + 1)),
    ('TotalWorkingYearsToJobLevelRatio', lambda df: df['TotalWorkingYears'] / (df['JobLevel'] + 1)),
    ('YearsAtCompanyToAgeRatio', lambda df: df['YearsAtCompany'] / (df['Age'] + 1)),
    ('YearsAtCompanyToYearsInCurrentRoleRatio', lambda df: df['YearsAtCompany'] / (df['YearsInCurrentRole'] + 1))
]

for name, func in engineered_features:
    data[name] = func(data)

numerical_columns = [x for x in data.select_dtypes(include=['int64', 'float64']).columns if x!= "Attrition"]

categorical_columns = data.select_dtypes(include=['object']).columns

processed_data = data.copy()

nominal_columns = [ 'Department', 'EducationField', 'JobRole', 'MaritalStatus']
processed_data = pd.get_dummies(processed_data, columns=nominal_columns, drop_first=True)

processed_data['Attrition'] = processed_data['Attrition'].map({'No': 0, 'Yes': 1})
processed_data['OverTime'] = processed_data['OverTime'].map({'No': 0, 'Yes': 1})
processed_data['Gender'] = processed_data['Gender'].map({'Male': 0, 'Female': 1})
processed_data['BusinessTravel'] = processed_data['BusinessTravel'].map({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})

X_processed = processed_data.drop(columns=["Attrition"], errors='ignore')
y_processed = processed_data["Attrition"]


In [ ]:
N_TRIALS = 50
import optuna
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold, train_test_split
from optuna.importance import get_param_importances

from sklearn.neural_network import MLPClassifier

numeric_transformer = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X_processed, y_processed, test_size=0.2, random_state=42, stratify=y_processed)

sampling_techniques = {
    "None": None,
    "SMOTE": SMOTE(random_state=42),
    "BorderlineSMOTE": BorderlineSMOTE(random_state=42),
    "SMOTETomek": SMOTETomek(random_state=42),
    "ADASYN": ADASYN(random_state=42),
}

models_and_params = {
    "MLP1": MLPClassifier,
    "MLP2": MLPClassifier,
    "MLP3": MLPClassifier,
    "MLP4": MLPClassifier,
   
}

combined_results = []

for sampling_name, sampler_type in sampling_techniques.items():
    for model_name, model_class in models_and_params.items():
        print(f"Optimizing {model_name}")

        def objective(trial, sampler):
            
            if model_name == "MLP1":
                model = MLPClassifier(
                    hidden_layer_sizes=trial.suggest_categorical('hidden_layer_sizes', [(128, 64, 32, 16)]),
                    activation=trial.suggest_categorical("activation", ["tanh", "relu"]),
                    solver=trial.suggest_categorical("solver", ["adam"]),
                    alpha=trial.suggest_float("alpha", 1e-5, 1e-2, log=True),
                    learning_rate_init=trial.suggest_float("learning_rate_init", 1e-4, 1e-1, log=True),
                    max_iter=1500,
                    random_state=42)

            elif model_name == "MLP2":
                model = MLPClassifier(
                    hidden_layer_sizes=trial.suggest_categorical('hidden_layer_sizes', [(256, 128, 64, 32, 16), (64, 32, 16), (32, 16)]),
                    activation=trial.suggest_categorical("activation", ["tanh", "relu"]),
                    solver=trial.suggest_categorical("solver", ["adam"]),
                    alpha=trial.suggest_float("alpha", 1e-5, 1e-2, log=True),
                    learning_rate_init=trial.suggest_float("learning_rate_init", 1e-4, 1e-1, log=True),
                    max_iter=1500,
                    random_state=42)

            elif model_name == "MLP3":
                model = MLPClassifier(
                    hidden_layer_sizes=trial.suggest_categorical('hidden_layer_sizes', [( 64,64, 64), (64, 64)]),
                    activation=trial.suggest_categorical("activation", ["tanh", "relu"]),
                    solver=trial.suggest_categorical("solver", ["adam"]),
                    alpha=trial.suggest_float("alpha", 1e-5, 1e-2, log=True),
                    learning_rate_init=trial.suggest_float("learning_rate_init", 1e-4, 1e-1, log=True),
                    max_iter=1500,
                    random_state=42)


            else:
                raise ValueError(f"Unknown model: {model_name}")


            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            f1_scores = []

            for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
                X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]

                preprocessor = ColumnTransformer(
                    transformers=[
                        ('num', numeric_transformer, numerical_columns)],
                    remainder='passthrough')

                if sampler:

                    X_train_cv, y_train_cv = sampler.fit_resample(X_train_cv, y_train_cv)
                    X_train_cv = preprocessor.fit_transform(X_train_cv)
                else:
                    X_train_cv = preprocessor.fit_transform(X_train_cv)
                
                X_val_cv = preprocessor.transform(X_val_cv)

                model.fit(X_train_cv, y_train_cv)
                y_pred_cv = model.predict(X_val_cv)

                f1 = f1_score(y_val_cv, y_pred_cv, average="weighted")
                f1_scores.append(f1)

                trial.report(f1, fold)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

            return np.mean(f1_scores)

        study = optuna.create_study(direction="maximize")
        study.optimize(lambda trial: objective(trial, sampler_type), n_trials=N_TRIALS)


        try:
            importance = get_param_importances(study)
        except:
            importance = None

        best_trial = study.best_trial
        best_params = best_trial.params

        model_params = best_params.copy()
        if model_name in ["KNN", "Gaussian Naive Bayes"]:
            best_model = model_class(**model_params)
        else:
            best_model = model_class(**model_params, random_state=42)

        preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_columns)],remainder='passthrough')
        if sampler_type:
            X_train_transformed = preprocessor.fit_transform(X_train)
            X_train_resampled, y_train_resampled = sampler_type.fit_resample(X_train_transformed, y_train)
        else:
            X_train_resampled = preprocessor.fit_transform(X_train)
            y_train_resampled = y_train


        best_model.fit(X_train_resampled, y_train_resampled)

        X_test_transformed = preprocessor.transform(X_test)

        y_pred_test = best_model.predict(X_test_transformed)
        y_prob_test = best_model.predict_proba(X_test_transformed)[:, 1]

        f1_weighted = f1_score(y_test, y_pred_test, average="weighted")
        precision_weighted = precision_score(y_test, y_pred_test, average="weighted")
        recall_weighted = recall_score(y_test, y_pred_test, average="weighted")

        roc_auc = roc_auc_score(y_test, y_prob_test)

        conf_matrix = confusion_matrix(y_test, y_pred_test)
     
        precision, recall, _ = precision_recall_curve(y_test, y_prob_test)
        pr_auc = auc(recall, precision)

        combined_results.append({
            "Model": model_name,
            "Sampling": sampling_name,
            "Validation F1-Score (CV)": best_trial.value,
            "Test F1-Score (Weighted)": f1_weighted,
            "Test Precision (Weighted)": precision_weighted,
            "Test Recall (Weighted)": recall_weighted,
            "Test ROC-AUC": roc_auc,
            "Test PR-AUC": pr_auc,
            "Best Parameters": best_params,
            "hyperparameter_importance": importance,
            "confusion_matrix": conf_matrix
        })


In [ ]:
results_df = pd.DataFrame(combined_results)

results_df.to_csv("mlp_results.csv")